# 1.0. Initialize

In [21]:

import numpy as np
from nibabel.viewers import OrthoSlicer3D
from nibabel import nifti1
import nibabel as nib
from matplotlib import pylab as plt

from ml_logic.data import load_nii_from_gcp

In [22]:
from google.cloud import storage
import os


client = storage.Client()

#get the Bucket Name from .env
bucket_name = os.getenv('BUCKET_NAME')
#set bucket
bucket = client.get_bucket(bucket_name)
#give the blob_name that you want to access
blob_name='BraTS20_Training_005_seg.nii' 
#get the blob(file)
blob = bucket.blob(blob_name)
# make a lockal cach folder
directory = "cache" 
parent_dir = "/users/paulhermanns/code/" #need to change the yaoyx001 to your username
# Path
path = os.path.join(parent_dir, directory)
  
# Create the directory
if not os.path.exists(path):
    os.makedirs(path)
    print("Directory '% s' created" % directory)

cache_file = os.path.join(path, blob_name)
#save the file in cache_folder

if not os.path.isfile(cache_file):
    blob.download_to_filename(cache_file)



# 1.1 Select Data

In [23]:
templist = []
for i in range (1,10):
    img = load_nii_from_gcp(f'BraTS20_Training_00{i}_seg.nii' ,path)
    np_ar = np.array(img.dataobj)
    templist.append(np_ar)
    

# print the img

print(templist)

templist

[array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ...,

[array([[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],
 
        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],
 
        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],
 
        ...,
 
        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],
 
        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],

# 1.2. First model pipeline construction

In [24]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# the target columns have to be updated, it's only the fremework
df['Grade'] = df['Grade'].apply(lambda x: 0 if x == 'HGG' else 1)

# the target columns have to be updated, it's only the fremework
X_train, X_test, y_train, y_test = train_test_split(df['preprocessed_segments'], df['Grade'], test_size=0.2, random_state=42)

logreg_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('logreg', LogisticRegression())
])

logreg_pipeline.fit(X_train.values.reshape(-1, 1), y_train)

y_pred = logreg_pipeline.predict(X_test.values.reshape(-1, 1))


print(classification_report(y_test, y_pred))


NameError: name 'df' is not defined

# Second Model

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import OneHotEncoder


def survival_encoding(x):
    if x <= 180.00:
        return 0
    elif x <= 515.00:
        return 1
    else:
        return 2

# the target columns have to be updated, it's only the fremework
df['Survival'] = df['Survival_days'].apply(survival_encoding)

# the target columns have to be updated, it's only the fremework
X_train, X_test, y_train, y_test = train_test_split(df[['Age', 'Grade']], df['Survival'], test_size=0.2, random_state=42)


numeric_transformer = Pipeline([
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline([
    ('onehot', OneHotEncoder())
])


preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, ['Age']),
        ('cat', categorical_transformer, ['Grade']) #it should already have been preprocesseso we don't need it
    ])

discretizer = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='uniform')


logreg_model = LogisticRegression(multi_class='multinomial', solver='lbfgs')


model_2_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('discretizer', discretizer),
    ('logreg', logreg_model)
])


model_2_pipeline.fit(X_train, y_train)



